In [ ]:
import dataclasses as dc
import os

import torch
import numpy as np
from PIL import Image

from pydrake.math import RollPitchYaw, RotationMatrix

import neural_renderer as nr
from simple_mesh import Mesh, make_fake_textures, intrinsic_matrix_from_fov

In [ ]:
np.set_printoptions(formatter={"float_kind": lambda x: f"{x:,.4f}"})
torch.set_grad_enabled(False)

In [ ]:
device = torch.device("cuda")
dtype = torch.float32

In [ ]:
width = 128
height = 128
fov_y = np.pi / 4
K = intrinsic_matrix_from_fov(width, height, fov_y)
K = torch.from_numpy(K).to(device, dtype).unsqueeze(0)

renderer = nr.Renderer(
    camera_mode='projection',
    projection_left_hand=False,
    image_size=width,
    light_direction=[-1, -1, -1],
    light_intensity_directional=0.5,
    background_color=[1.0, 1.0, 1.0],
)

In [ ]:
vertices, faces, textures = nr.load_obj(
    "textured_cylinder.obj", normalization=False, load_texture=True,
)

In [ ]:
R_CW = np.eye(3)
p_CW = np.array([0, 0, 10])

for rot_y in np.arange(0, 360, 15):
    print(rot_y)
    R_WWh = RollPitchYaw([0, np.deg2rad(rot_y), 0]).ToRotationMatrix().matrix()
# for rot_x in np.arange(0, 360, 15):
#     print(rot_x)
#     R_WWh = RollPitchYaw([np.deg2rad(rot_x), 0, 0]).ToRotationMatrix().matrix()

    v_hack = vertices @ torch.from_numpy(R_WWh).to(device, dtype).T

    image = renderer.render_rgb(
        v_hack.unsqueeze(0),
        faces.unsqueeze(0),
        textures=textures.unsqueeze(0),
        R=torch.from_numpy(R_CW).to(device, dtype).unsqueeze(0),
        t=torch.from_numpy(p_CW).to(device, dtype).unsqueeze(0),
        K=K,
    ).squeeze(0).cpu().numpy().transpose(1, 2, 0)

    rgb = (image * 255).astype(np.uint8)
    display(Image.fromarray(rgb))